In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.interpolate import interp1d
from tqdm import tqdm
import gc

class SchwarzschildRayTracerBatch:
    def __init__(self, bh_mass=1.0, device=None):
        self.G = 1.0
        self.c = 1.0
        self.E = 1.0
        self.M = bh_mass
        self.rs = 2 * self.G * self.M / self.c**2
        self.b_max = 15 * self.rs
        self.device = device if device else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

    def V(self, r, b):
        return (b**2 / (2 * r**2) - self.M * b**2 / r**3)

    def dV_dr(self, r, b):
        return (-b**2 / r**3 + 3 * self.M * b**2 / r**4)

    def system(self, y, b):
        r = y[:, 0]
        phi = y[:, 1]
        dr_dtau = y[:, 2]

        d2r_dtau2 = -self.dV_dr(r, b)
        dphi_dtau = b / r**2

        dydtau = torch.stack([dr_dtau, dphi_dtau, d2r_dtau2], dim=1)
        return dydtau

    def rkf45_step_batch(self, f, t, y, h, b):
        a = [0, 1/4, 3/8, 12/13, 1, 1/2]
        b_ = [
            [],
            [1/4],
            [3/32, 9/32],
            [1932/2197, -7200/2197, 7296/2197],
            [439/216, -8, 3680/513, -845/4104],
            [-8/27, 2, -3544/2565, 1859/4104, -11/40]
        ]
        c4 = [25/216, 0, 1408/2565, 2197/4104, -1/5, 0]
        c5 = [16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55]

        k1 = f(y, b)
        k2 = f(y + h * (b_[1][0] * k1), b)
        k3 = f(y + h * (b_[2][0] * k1 + b_[2][1] * k2), b)
        k4 = f(y + h * (b_[3][0] * k1 + b_[3][1] * k2 + b_[3][2] * k3), b)
        k5 = f(y + h * (b_[4][0] * k1 + b_[4][1] * k2 + b_[4][2] * k3 + b_[4][3] * k4), b)
        k6 = f(y + h * (b_[5][0] * k1 + b_[5][1] * k2 + b_[5][2] * k3 + b_[5][3] * k4 + b_[5][4] * k5), b)

        y4 = y + h * (c4[0]*k1 + c4[2]*k3 + c4[3]*k4 + c4[4]*k5)
        y5 = y + h * (c5[0]*k1 + c5[2]*k3 + c5[3]*k4 + c5[4]*k5 + c5[5]*k6)

        error = torch.norm(y5 - y4, dim=1)
        return y5, error

    def solve_batch(self, b_array, r0=100.0, phi0=0.0, tau_span=(0, 140), h=0.1, tol=1e-5, N=1000):
        b = torch.tensor(b_array, dtype=torch.float32, device=self.device)
        batch_size = b.shape[0]
        r0_t = torch.full((batch_size,), r0, dtype=torch.float32, device=self.device)
        phi0_t = torch.full((batch_size,), phi0, dtype=torch.float32, device=self.device)

        V0 = self.V(r0_t, b)
        dr0 = -torch.sqrt(torch.clamp(self.E**2 - 2 * V0, min=0.0))

        y = torch.stack([r0_t, phi0_t, dr0], dim=1)
        t = tau_span[0]
        tf = tau_span[1]

        ys = [y.cpu()]
        active = torch.ones(batch_size, dtype=torch.bool, device=self.device)

        m=int((tf-t)/h/N)
        n=0

        with tqdm(total=N, desc="Renderizando", unit="px") as pbar:
            while t < tf and active.any():
                y_next, error = self.rkf45_step_batch(self.system, t, y, h, b)
                y = torch.where(active.unsqueeze(1), y_next, y)
                active = active & (y[:, 0] > self.rs)
                n +=1
                if n==m:
                    ys.append(y.cpu())
                    n=0
                    pbar.update(1)
                t += h

        return torch.stack(ys)

    def render(self, N_x=100, N_y=100, gamma_deg=20, save_path=None):
        ancho, alto = N_x, N_y
        imagen = Image.new('RGB', (ancho, alto), color=(0,0,0))
        pixeles = imagen.load()

        R= self.matriz_rotacion(0,0, gamma_deg)
        R2= self.matriz_rotacion(0,0, -gamma_deg)

        xs = np.linspace(-1, 0, ancho//2)
        ys = np.linspace(-1, 0 , alto//2)
        xv, yv = np.meshgrid(xs, ys)

        b_vals = self.b_max * np.sqrt(xv.flatten()**2 + yv.flatten()**2)
        N=500

        sol_batch = self.solve_batch(b_vals, r0=100*self.rs, tau_span=(0,2000), h=0.1, N=N)
        rs = self.rs

        r_vals = sol_batch[:, :, 0].numpy()
        phi_vals = sol_batch[:, :, 1].numpy()

        del sol_batch
        gc.collect()


        x1 =np.zeros(N)
        y1 =np.zeros(N)
        z1 =np.zeros(N)

        idx = 0
        with tqdm(total=ancho//2, desc="Renderizando", unit="px") as pbar:
            for i in range(ancho//2):
                for j in range(alto//2):
                    b = self.b_max * np.sqrt(xv[j,i]**2 + yv[j,i]**2)

                    r_traj = r_vals[:, idx]
                    phi_traj = phi_vals[:, idx]
                    #print(len(r_traj))

                    theta = np.arctan2(yv[j,i], xv[j,i])

                    tau_original = np.linspace(0, len(r_traj)-1, len(r_traj))  # o el vector real de tau si tienes

                    indices_validos = np.where((r_traj > 6) & (r_traj < 20))[0]

                    if len(indices_validos) > 1:

                        tau_sub = tau_original[indices_validos]
                        r_sub = r_traj[indices_validos]
                        phi_sub = phi_traj[indices_validos]

                        if len(tau_sub) >= 4:  # mínimo 4 puntos para 'cubic'
                            interp_r = interp1d(tau_sub, r_sub, kind='cubic')
                            interp_phi = interp1d(tau_sub, phi_sub, kind='cubic')
                        else:
                            interp_r = interp1d(tau_sub, r_sub, kind='linear')
                            interp_phi = interp1d(tau_sub, phi_sub, kind='linear')

                        tau_fino = np.linspace(tau_sub[0], tau_sub[-1], num=300)

                        r_interp = interp_r(tau_fino)
                        phi_interp = interp_phi(tau_fino)

                        # Calcular coordenadas 3D interpoladas
                        x_interp = r_interp * np.cos(theta) * np.sin(phi_interp)
                        y_interp = r_interp * np.sin(theta) * np.sin(phi_interp)
                        z_interp = r_interp * np.cos(phi_interp)

                        points = np.vstack((x_interp, y_interp, z_interp)).T  # shape (N, 3)
                        rotated_points = points @ R.T  # shape (N, 3)

                        x1 = rotated_points[:, 0]
                        y1 = rotated_points[:, 1]
                        z1 = rotated_points[:, 2]

                        Rmin=4.32*rs
                        Rmax=9.16*rs

                        A = (np.sqrt(x1**2 + z1**2) > Rmin) & (np.sqrt(x1**2 + z1**2) < Rmax)
                        B = np.abs(np.arctan(y1 / np.sqrt(x1**2 + z1**2))) < 0.01

                        if np.any(A & B):
                            arg=np.where(A & B)[0][0]
                            x1=x1[arg]
                            y1=y1[arg]
                            z1=z1[arg]
                                    
                            r=np.sqrt(x1**2 + z1**2)
                            argumento=(r-Rmin)/(Rmax-Rmin)

                            phi= np.arctan2(z1,x1)

                            f=np.abs(np.cos(argumento*40*np.pi))*(1-argumento)
                                    
                            f_dop=Rmin*(1-0.75*np.cos(phi)*np.cos(gamma_deg/180*np.pi))*self.c*np.sqrt(self.rs/r)*(r/(r-rs))
                                    
                            Red=int((150*f+30*f_dop+75))
                            Green=int(30*f+8*f_dop+5)
                            Blue=int(20*f_dop*f)

                            pixeles[i, j] = (Red, Green, Blue) 

                            f_dop=Rmin*(1+0.75*np.cos(phi)*np.cos(gamma_deg/180*np.pi))*self.c*np.sqrt(self.rs/r)*(r/(r-rs))
                                    
                            Red=int((150*f+30*f_dop+75))
                            Green=int(30*f+8*f_dop+5)
                            Blue=int(20*f_dop*f)

                            pixeles[ancho-i-1, j] = (Red, Green, Blue) 

                        else:
                            pixeles[i, j] = (0, 0, 0)
                            pixeles[ancho-i-1, j]=(0,0,0)
                            
                        y_interp=-y_interp
                            
                        points = np.vstack((x_interp, y_interp, z_interp)).T  # shape (N, 3)
                        rotated_points = points @ R.T  # shape (N, 3)

                        x1 = rotated_points[:, 0]
                        y1 = rotated_points[:, 1]
                        z1 = rotated_points[:, 2]

                        Rmin=4.32*rs
                        Rmax=9.16*rs

                        A = (np.sqrt(x1**2 + z1**2) > Rmin) & (np.sqrt(x1**2 + z1**2) < Rmax)
                        B = np.abs(np.arctan(y1 / np.sqrt(x1**2 + z1**2))) < 0.01

                        if np.any(A & B):
                            arg=np.where(A & B)[0][0]
                            x1=x1[arg]
                            y1=y1[arg]
                            z1=z1[arg]
                            
                            r=np.sqrt(x1**2 + z1**2)
                            argumento=(r-Rmin)/(Rmax-Rmin)

                            phi= np.arctan2(z1,x1)

                            f=np.abs(np.cos(argumento*40*np.pi))*(1-argumento)
                                
                            f_dop=Rmin*(1+0.75*np.cos(phi)*np.cos(gamma_deg/180*np.pi))*self.c*np.sqrt(self.rs/r)*(r/(r-rs))
                                
                            Red=int((220*f+30*f_dop+5))
                            Green=int(30*f+8*f_dop+5)
                            Blue=int(20*f_dop*f)

                            pixeles[i, alto-j-1] = (Red, Green, Blue) 

                            f_dop=Rmin*(1+0.75*np.cos(phi)*np.cos(gamma_deg/180*np.pi))*self.c*np.sqrt(self.rs/r)*(r/(r-rs))
                                
                            Red=int((150*f+30*f_dop+75))
                            Green=int(30*f+8*f_dop+5)
                            Blue=int(20*f_dop*f)

                            pixeles[ancho-i-1, alto-j-1] = (Red, Green, Blue) 

                        else:
                            pixeles[i, alto-j-1] = (0, 0, 0)
                            pixeles[ancho-i-1, alto-j-1]=(0,0,0)
                    else:
                        pixeles[i, j] = (0, 0, 0)
                        pixeles[ancho-i-1, j] = (0, 0, 0)
                        pixeles[i, alto-j-1] = (0, 0, 0)
                        pixeles[ancho-i-1, alto-j-1]=(0,0,0)
                    idx += 1
                pbar.update(1)

        imagen.save("../Multimedia/Imagen_disco_de_acreción.png")

    def matriz_rotacion(self, theta_deg=0, phi_deg=0, gamma_deg=0):
        theta = np.deg2rad(theta_deg)
        phi = np.deg2rad(phi_deg)
        gamma= np.deg2rad(gamma_deg)
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])
        Ry = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(gamma), np.sin(gamma)],
            [0, -np.sin(gamma), np.cos(gamma)]
        ])
        return Rz @ Ry @ Rx

rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)
rt_batch.render(N_x=300, N_y=300)

Renderizando: 100%|██████████| 150/150 [00:03<00:00, 44.91px/s]


In [ ]:
import cupy as cp
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.interpolate import interp1d
from tqdm import tqdm
import imageio
import gc
import pandas as pd

class SchwarzschildRayTracerBatch:
    def __init__(self, bh_mass=1.0, device=None):
        self.G = 1.0
        self.c = 1.0
        self.E = 1.0
        self.M = bh_mass
        self.rs = 2 * self.G * self.M / self.c**2
        self.b_max = 15 * self.rs
        self.device = device if device else (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

    def V(self, r, b):
        return (b**2 / (2 * r**2) - self.M * b**2 / r**3)

    def dV_dr(self, r, b):
        return (-b**2 / r**3 + 3 * self.M * b**2 / r**4)

    def system(self, y, b):
        r = y[:, 0]
        phi = y[:, 1]
        dr_dtau = y[:, 2]

        d2r_dtau2 = -self.dV_dr(r, b)
        dphi_dtau = b / r**2

        dydtau = torch.stack([dr_dtau, dphi_dtau, d2r_dtau2], dim=1)
        return dydtau

    def rkf45_step_batch(self, f, t, y, h, b):
        a = [0, 1/4, 3/8, 12/13, 1, 1/2]
        b_ = [
            [],
            [1/4],
            [3/32, 9/32],
            [1932/2197, -7200/2197, 7296/2197],
            [439/216, -8, 3680/513, -845/4104],
            [-8/27, 2, -3544/2565, 1859/4104, -11/40]
        ]
        c4 = [25/216, 0, 1408/2565, 2197/4104, -1/5, 0]
        c5 = [16/135, 0, 6656/12825, 28561/56430, -9/50, 2/55]

        k1 = f(y, b)
        k2 = f(y + h * (b_[1][0] * k1), b)
        k3 = f(y + h * (b_[2][0] * k1 + b_[2][1] * k2), b)
        k4 = f(y + h * (b_[3][0] * k1 + b_[3][1] * k2 + b_[3][2] * k3), b)
        k5 = f(y + h * (b_[4][0] * k1 + b_[4][1] * k2 + b_[4][2] * k3 + b_[4][3] * k4), b)
        k6 = f(y + h * (b_[5][0] * k1 + b_[5][1] * k2 + b_[5][2] * k3 + b_[5][3] * k4 + b_[5][4] * k5), b)

        y4 = y + h * (c4[0]*k1 + c4[2]*k3 + c4[3]*k4 + c4[4]*k5)
        y5 = y + h * (c5[0]*k1 + c5[2]*k3 + c5[3]*k4 + c5[4]*k5 + c5[5]*k6)

        error = torch.norm(y5 - y4, dim=1)
        return y5, error

    def solve_batch(self, b_array, r0=100.0, phi0=0.0, tau_span=(0, 140), h=0.1, tol=1e-5, N=1000):
        b = torch.tensor(b_array, dtype=torch.float32, device=self.device)
        batch_size = b.shape[0]
        r0_t = torch.full((batch_size,), r0, dtype=torch.float32, device=self.device)
        phi0_t = torch.full((batch_size,), phi0, dtype=torch.float32, device=self.device)

        V0 = self.V(r0_t, b)
        dr0 = -torch.sqrt(torch.clamp(self.E**2 - 2 * V0, min=0.0))

        y = torch.stack([r0_t, phi0_t, dr0], dim=1)
        t = tau_span[0]
        tf = tau_span[1]

        ys = [y.cpu()]
        active = torch.ones(batch_size, dtype=torch.bool, device=self.device)

        m=int((tf-t)/h/N)
        n=0
        with tqdm(total=N, desc="Calculando trayectorias", unit="Tr") as pbar:
          while t < tf and active.any():
            #while error>
            y_next, error = self.rkf45_step_batch(self.system, t, y, h, b)

            y = torch.where(active.unsqueeze(1), y_next, y)
            active = active & (y[:, 0] > self.rs)
            n +=1
            if n==m:
              ys.append(y.cpu())
              n=0
              pbar.update(1)
            t += h

        return torch.stack(ys)

    def render(self, N_x=100, N_y=100, gamma_deg=20, ti=0, tf=2000):
        ancho, alto = N_x, N_y

        xs = np.linspace(-1, 0, ancho//2)
        ys = np.linspace(-1, 0, alto//2)
        xv, yv = np.meshgrid(xs, ys)

        b_vals = self.b_max * np.sqrt(xv.flatten()**2 + yv.flatten()**2)
        N=500

        sol_batch = self.solve_batch(b_vals, r0=100 * self.rs, tau_span=(ti, tf), h=0.1, N=N)

        r_vals = sol_batch[:, :, 0].numpy()   # shape: (num_tiempo, num_rayos)
        phi_vals = sol_batch[:, :, 1].numpy()

        del sol_batch
        gc.collect()
        
        rs = self.rs

        x1 =np.zeros(N)
        y1 =np.zeros(N)
        z1 =np.zeros(N)

        gamma_deg_vals=np.linspace(0,180, 36)
        with tqdm(total=len(gamma_deg_vals), desc="Renderizando", unit="Imágenes") as pbar:
          with imageio.get_writer("../Multimedia/rotacion.mp4", fps=10) as writer:
            for gamma_deg in gamma_deg_vals:
                R= self.matriz_rotacion(0,0, gamma_deg)
                imagen = Image.new('RGB', (ancho, alto), color=(0,0,0))
                imagen2 = Image.new('RGB', (ancho, alto), color=(0,0,0))
                pixeles = imagen.load()
                pixeles2= imagen2.load()
                idx=0

                for i in range(ancho//2):
                    for j in range(alto//2):
                        b = self.b_max * np.sqrt(xv[j,i]**2 + yv[j,i]**2)

                        r_traj = r_vals[:, idx]
                        phi_traj = phi_vals[:, idx]
                        #print(len(r_traj))

                        theta = np.arctan2(yv[j,i], xv[j,i])

                        tau_original = np.linspace(0, len(r_traj)-1, len(r_traj))  # o el vector real de tau si tienes

                        indices_validos = np.where((r_traj > 6) & (r_traj < 20))[0]

                        if len(indices_validos) > 1:

                            tau_sub = tau_original[indices_validos]
                            r_sub = r_traj[indices_validos]
                            phi_sub = phi_traj[indices_validos]

                            if len(tau_sub) >= 4:  # mínimo 4 puntos para 'cubic'
                                interp_r = interp1d(tau_sub, r_sub, kind='cubic')
                                interp_phi = interp1d(tau_sub, phi_sub, kind='cubic')
                            else:
                                interp_r = interp1d(tau_sub, r_sub, kind='linear')
                                interp_phi = interp1d(tau_sub, phi_sub, kind='linear')

                            tau_fino = np.linspace(tau_sub[0], tau_sub[-1], num=300)

                            r_interp = interp_r(tau_fino)
                            phi_interp = interp_phi(tau_fino)

                            # Calcular coordenadas 3D interpoladas
                            x_interp = r_interp * np.cos(theta) * np.sin(phi_interp)
                            y_interp = r_interp * np.sin(theta) * np.sin(phi_interp)
                            z_interp = r_interp * np.cos(phi_interp)

                            points = np.vstack((x_interp, y_interp, z_interp)).T  # shape (N, 3)
                            rotated_points = points @ R.T  # shape (N, 3)

                            x1 = rotated_points[:, 0]
                            y1 = rotated_points[:, 1]
                            z1 = rotated_points[:, 2]

                            Rmin=4.32*rs
                            Rmax=9.16*rs

                            A = (np.sqrt(x1**2 + z1**2) > Rmin) & (np.sqrt(x1**2 + z1**2) < Rmax)
                            B = np.abs(np.arctan(y1 / np.sqrt(x1**2 + z1**2))) < 0.01

                            if np.any(A & B):
                                arg=np.where(A & B)[0][0]
                                x1=x1[arg]
                                y1=y1[arg]
                                z1=z1[arg]
                                
                                r=np.sqrt(x1**2 + z1**2)
                                argumento=(r-Rmin)/(Rmax-Rmin)

                                phi= np.arctan2(z1,x1)

                                #t=np.sin(4*phi+np.cos(argumento*4* np.pi))

                                f=np.abs(np.cos(argumento*14*np.pi))*(1-argumento)#*t
                                
                                f_dop=Rmin*(1+0.75*np.cos(phi)*np.cos(gamma_deg/180*np.pi))*self.c*np.sqrt(self.rs/r)*(r/(r-rs))
                                
                                Red=int((220*f+30*f_dop+5))
                                Green=int(50*f+8*f_dop+5)
                                Blue=int(20*f_dop*f)

                                pixeles[i, j] = (Red, Green, Blue) 

                                f_dop=Rmin*(1-0.75*np.cos(phi)*np.cos(gamma_deg/180*np.pi))*self.c*np.sqrt(self.rs/r)*(r/(r-rs))
                                
                                Red=int((220*f+30*f_dop+5))
                                Green=int(50*f+8*f_dop+5)
                                Blue=int(20*f_dop*f)

                                pixeles[ancho-i-1, j] = (Red, Green, Blue) 

                            else:
                                pixeles[i, j] = (0, 0, 0)
                                pixeles[ancho-i-1, j]=(0,0,0)
                            
                            y_interp=-y_interp
                            
                            points = np.vstack((x_interp, y_interp, z_interp)).T  # shape (N, 3)
                            rotated_points = points @ R.T  # shape (N, 3)

                            x1 = rotated_points[:, 0]
                            y1 = rotated_points[:, 1]
                            z1 = rotated_points[:, 2]

                            Rmin=4.32*rs
                            Rmax=9.16*rs

                            A = (np.sqrt(x1**2 + z1**2) > Rmin) & (np.sqrt(x1**2 + z1**2) < Rmax)
                            B = np.abs(np.arctan(y1 / np.sqrt(x1**2 + z1**2))) < 0.01

                            if np.any(A & B):
                                arg=np.where(A & B)[0][0]
                                x1=x1[arg]
                                y1=y1[arg]
                                z1=z1[arg]
                                
                                r=np.sqrt(x1**2 + z1**2)
                                argumento=(r-Rmin)/(Rmax-Rmin)

                                phi= np.arctan2(z1,x1)

                                f=np.abs(np.cos(argumento*14*np.pi))*(1-argumento)
                                
                                f_dop=Rmin*(1+0.75*np.cos(phi)*np.cos(gamma_deg/180*np.pi))*self.c*np.sqrt(self.rs/r)*(r/(r-rs))
                                
                                Red=int((220*f+30*f_dop+5))
                                Green=int(50*f+8*f_dop+5)
                                Blue=int(20*f_dop*f)

                                pixeles[i, alto-j-1] = (Red, Green, Blue) 

                                f_dop=Rmin*(1-0.75*np.cos(phi)*np.cos(gamma_deg/180*np.pi))*self.c*np.sqrt(self.rs/r)*(r/(r-rs))
                                
                                Red=int((220*f+30*f_dop+5))
                                Green=int(50*f+8*f_dop+5)
                                Blue=int(20*f_dop*f)

                                pixeles[ancho-i-1, alto-j-1] = (Red, Green, Blue) 

                            else:
                                pixeles[i, alto-j-1] = (0, 0, 0)
                                pixeles[ancho-i-1, alto-j-1]=(0,0,0)
                        else:
                            pixeles[i, j] = (0, 0, 0)
                            pixeles[ancho-i-1, j] = (0, 0, 0)
                            pixeles[i, alto-j-1] = (0, 0, 0)
                            pixeles[ancho-i-1, alto-j-1]=(0,0,0)

                        idx += 1
                pbar.update(1)
                frame = np.array(imagen)
                writer.append_data(frame)

    def matriz_rotacion(self, theta_deg=0, phi_deg=0, gamma_deg=0):
        theta = np.deg2rad(theta_deg)
        phi = np.deg2rad(phi_deg)
        gamma= np.deg2rad(gamma_deg)
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta),  np.cos(theta), 0],
            [0, 0, 1]
        ])
        Ry = np.array([
            [np.cos(phi), 0, np.sin(phi)],
            [0, 1, 0],
            [-np.sin(phi), 0, np.cos(phi)]
        ])
        Rx = np.array([
            [1, 0, 0],
            [0, np.cos(gamma), np.sin(gamma)],
            [0, -np.sin(gamma), np.cos(gamma)]
        ])
        return Rz @ Ry @ Rx

rt_batch = SchwarzschildRayTracerBatch(bh_mass=1.0)
#rt_batch.render(N_x=112, N_y=112)
rt_batch.render(N_x=512, N_y=512)

Renderizando: 100%|██████████| 36/36 [06:28<00:00, 10.79s/Imágenes]


Por si en algún momento se prefiere hacer mediante un fichero

In [ ]:
    def read_or_write(self, N_x=100, N_y=100, b_vals):
        filename = "../datos/ray_tracing_Schwarzschild_data.csv"

        try:
            # Intenta leer el archivo, omitiendo las primeras 7 líneas de información
            df = pd.read_csv(filename, skiprows=7)

            # Verificación del número de píxeles esperados
            N_pixeles_esperados = N_x * N_y // 4  # Segundo cuadrante (x < 0, y > 0)
            pixeles_en_archivo = df["pixel_id"].nunique()

            if pixeles_en_archivo != N_pixeles_esperados:
                raise ValueError("Número de píxeles en archivo no coincide")
            
            df_pivot_r = df.pivot(index="tau", columns="pixel_id", values="r")
            df_pivot_phi = df.pivot(index="tau", columns="pixel_id", values="phi")

            r_vals = df_pivot_r.to_numpy()  # shape: (N_t, N_pixeles)
            phi_vals = df_pivot_phi.to_numpy()

            # Limpiar memoria
            del df, df_pivot_r, df_pivot_phi
            gc.collect()

        except (FileNotFoundError, ValueError, pd.errors.ParserError) as e:
            print(f"Archivo no válido o no encontrado. Se volverá a calcular. Detalles: \n {e}")

            # Calcular los datos
            sol_batch = self.solve_batch(b_vals, r0=100 * self.rs, tau_span=(ti, tf), h=0.1, N=N)

            r_vals = sol_batch[:, :, 0].numpy()   # shape: (num_tiempo, num_rayos)
            phi_vals = sol_batch[:, :, 1].numpy()

            sol_np = sol_batch.cpu().numpy()
            N_t, N_pixeles, _ = sol_np.shape

            rows = []
            for idx in range(N_pixeles):
                for t_idx in range(N_t):
                    r = sol_np[t_idx, idx, 0]
                    phi = sol_np[t_idx, idx, 1]
                    #dr = sol_np[t_idx, idx, 2]
                    rows.append([idx, r, phi])

            df = pd.DataFrame(rows, columns=["pixel_id", "r", "phi"])

            os.makedirs(os.path.dirname(filename), exist_ok=True)

            with open(filename, "w") as f:
                f.write("Ray tracing data (Schwarzschild metric)\n")
                f.write("Archivo generado por Schwarzschild_Black_Hole_Accretion_disk_mp4.ipynb\n")
                f.write("Contiene las trayectorias (r, phi, dr/dtau) para cada rayo (pixel_id)\n")
                f.write("Región: Segundo cuadrante (x < 0, y > 0)\n")
                f.write(f"Número de píxeles: {N_pixeles}\n")
                f.write(f"Número de pasos temporales: {N_t}\n")
                f.write("Columnas: pixel_id, r, phi \n")
                f.write("------------------------------------------------------------------------------------\n")
                df.to_csv(f, index=False)

            del sol_batch
            gc.collect()
        